# Modelado de tópicos con BERTopic


Para compreder acerca de qué temas se habla en un conjunto de documentos, es necesario realizar un análisis de tópicos. El análisis de tópicos es una técnica de minería de texto que permite identificar los temas principales que se tratan en un conjunto de documentos. Exiten una gran cantidad de algoritmos que permiten realizar este análisis, entre ellos destacan *Latent Dirichlet Allocation, Latent Semantic Analysis, Non-negative Matrix Factorization*, entre otros.

Siendo la mayoria de estos métodos basados en la representación de los documentos como vectores de palabras, es decir, se representan los documentos como vectores de frecuencias de palabras, lo que implica que se pierde la información de contexto de las palabras.

En el año 2022 se lanzó [BERTopic](https://arxiv.org/abs/2203.05794), un algoritmo de análisis de tópicos basado en la representación de los documentos como vectores de palabras, pero utilizando la representación de palabras de cualquier modelo transformer de lenguaje (como BERT), a modo que la información contextual contenida dentro de cada texto sea aprovechada. BERTopic es un algoritmo de análisis de tópicos auto-,semi- y supervisado, lo que significa que no es necesario etiquetar todos los documentos para realizar el análisis de tópicos.

El modelo de BERTopic permite agrupar textos cuya semántica es similar y provee de una jerarquía de palabras simples o compuestas para cada grupo de textos. Esta jerarquía de palabras permite al usuario asociar un tema o conjunto de temas a cada grupo.
El funcionamiento de este modelo o algoritmo se divide en tres etapas:

* **Generación y extracción de embbedings por documento**: En esta etapa se utiliza un modelo transformer de lenguaje para generar los embeddings de cada documento. Los embeddings son vectores de números reales que representan la información contextual de cada documento.

* **Agrupamiento de los vectores de documentos en grupos semánticamente similares**: En esta etapa se utiliza un algoritmo de clustering para agrupar los vectores de documentos en grupos semánticamente similares. El algoritmo de clustering utilizado es DBSCAN, el cual permite identificar grupos de documentos que se encuentran cercanos entre sí y que no se encuentran cercanos a los grupos de otros documentos.

* **Creación de la representación de palabras o tópicos para cada grupo**: Para asignar a cada grupo un conjunto de palabras claves, los textos asignados a cada grupo son considerados como un único documento, para el cual se mide la importancia de las palabras contenidas a partir de la frecuencia de aparición. Esto se realiza gracias a la medida estadística conocida como *Term frequency – Inverse document frequency* (Tf-Idf). De esta manera, a partir de las palabras más importantes de cada grupo se obtiene una descripción del tópico.

En este notebook se presenta un ejemplo de aplicación de BERTopic semisupervisado para el análisis de tópicos de un conjunto de documentos. Se presentarán algunas gráficas que permiten visualizar algunos hitos dentro de los datos usados.

In [1]:
#Imports

from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')
nltk.download('punkt')
from umap import UMAP

[nltk_data] Downloading package stopwords to /home/pedri0/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/pedri0/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Carga de datos etiquetados y no etiquetados sobre migración en twitter

Se leerán los datos para modelado de topicos a través del modelo BERTopic, el cual será entrenado de manera semisupervisada. El conjunto de datos contiene etiquetas para los temas covid (477), servicios públicos (465), género (464), seguridad/crimen (429), salud (188) y empleo (427). El conjunto de datos contiene 10,000 tweets. Los textos no etiquetados contienen la etiqueta -1.

In [50]:
#read data
data = pd.read_csv('assets/data/bertopic_train.csv')
#convert string labels to int labels
target_label = {'covid': 0, 'servpub':1, 'genero':2, 'crimen':3, 'empleo':4, 'salud':5, '-1':int(-1)}
data['target'] = data.label.apply(lambda x: target_label[x])
data = data.sample(frac=0.2).reset_index(drop=True)

### Configuración del modelo BERTopic

In [55]:
#add our stop word through countvectorizer model, set to 2 the limit of n-grams
# with min_df: ignore terms that have a document frequency strictly lower than the given threshold (5)
vectorizer_model = CountVectorizer(stop_words=stopwords.words('spanish'), ngram_range=(1,2), min_df=1)

#use paraphrase-multilingual-mpnet-base-v2 as embedding model. This model is the most accurate for multilingual
#diversity is a parameter in BERTopic to diversity words in each topic such that we limit the number of 
#duplicate words we find in each topic. This is done using an algorithm called Maximal Marginal Relevance 
#which compares word embeddings with the topic embedding.
#We do this by specifying a value between 0 and 1, with 0 being not at all diverse and 1 being completely diverse
#min_topic_size is used to specify what the minimum size of a topic can be. 
#The lower this value the more topics are created.
#calculate_probabilities lets you calculate the probabilities of each topic to each document.
topic_model = BERTopic(embedding_model='paraphrase-multilingual-MiniLM-L12-v2', verbose=True,
                      vectorizer_model=vectorizer_model, diversity=0.3, min_topic_size=5,
                      calculate_probabilities=False)

In [56]:
#fit the model using semisupervised topic modeling and infer clusters, probabilities
topics = topic_model.fit_transform(data.text.to_list(), y=data.target.tolist())

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

2022-10-14 16:33:52,296 - BERTopic - Transformed documents to Embeddings
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2022-10-14 16:34:02,962 - BERTopic - Reduced dimensionality
2022-10-14 16:34:03,153 - BERTopic - Clustered reduced embeddings


In [57]:
#save inferred clusters of topics
data['cluster_topic'] = topics[0]

In [58]:
data.cluster_topic.value_counts()

-1     666
 0     114
 1     105
 2      94
 3      79
 4      55
 5      54
 6      52
 7      48
 8      47
 9      37
 10     35
 12     31
 11     31
 13     31
 14     30
 15     28
 16     27
 17     25
 18     24
 19     21
 20     19
 21     18
 22     18
 23     16
 24     15
 25     14
 28     13
 26     13
 27     13
 29     12
 30     12
 35     11
 31     11
 33     11
 32     11
 34     11
 38     10
 36     10
 37     10
 39      9
 42      8
 43      8
 41      8
 40      8
 44      8
 45      8
 46      7
 48      7
 47      7
 49      7
 50      6
 52      6
 51      6
 54      5
 55      5
 53      5
Name: cluster_topic, dtype: int64

In [59]:
#get dataframe with info about clusters from fitted model
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,666,-1_migrantes_inmigrantes_venezolanos_inmigracion
1,0,114,0_venezolanos_migrantes venezolanos_venezolana...
2,1,105,1_trabajo_empleo_desempleo_trabajadores
3,2,94,2_racista_racistas_negros_racismo
4,3,79,3_chile_chilenos_uruguay_argentina
5,4,55,4_apoyo_migrantes_migracion_apoyo migrantes
6,5,54,5_inmigrantes ilegales_ilegales_delincuentes_i...
7,6,52,6_migracion_pacto_pacto migratorio_gobierno
8,7,48,7_venezolanos_delincuentes venezolanos_venezol...
9,8,47,8_fronteras_cierre fronteras_cerrar fronteras_...


In [60]:
#get top 10 words per topic
topic_model.get_topics()

{-1: [('migrantes', 0.010131560160920173),
  ('inmigrantes', 0.007954665820730242),
  ('venezolanos', 0.007271173322471336),
  ('inmigracion', 0.0047090467928070714),
  ('migrante', 0.004690835661777644),
  ('mexico', 0.004083520080672387),
  ('ilegales', 0.003781216026435863),
  ('caravana', 0.003509960137426222),
  ('inmigrante', 0.0034206451449695858),
  ('migratoria', 0.00331592498238623)],
 0: [('venezolanos', 0.032568576707118525),
  ('migrantes venezolanos', 0.018753123563003904),
  ('venezolana', 0.012760919570526767),
  ('inmigrantes venezolanos', 0.009452647062884836),
  ('migracion venezolana', 0.0093921272643424),
  ('refugiados venezolanos', 0.009143018206162225),
  ('venezolano', 0.00784663219654577),
  ('migracion venezolanos', 0.006680512111826332),
  ('venezolanos colombia', 0.005592221199530613),
  ('personas venezolanas', 0.00548851567220626)],
 1: [('trabajo', 0.0257993974090455),
  ('empleo', 0.013977109824690194),
  ('desempleo', 0.012926353832764853),
  ('trabaja

In [16]:
#save model
topic_model.save("bertopic_20news.model")

/opt/conda/lib/python3.8/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [61]:
topic_model.visualize_barchart()

In [63]:
topic_model.visualize_hierarchy()

### Búsqueda de tópicos por palabra clave

In [74]:
similar_topics, similarity = topic_model.find_topics("coivd", top_n=5)
topic_model.get_topic(similar_topics[1])

[('migracion', 0.017281224006464452),
 ('pacto', 0.016027398789979344),
 ('pacto migratorio', 0.015974400687477817),
 ('gobierno', 0.015321652063977636),
 ('migratorio', 0.014383584855190729),
 ('politica', 0.009284416181367593),
 ('ciudadanos', 0.009158513594273911),
 ('inmigracion', 0.008881722159203263),
 ('leyes', 0.008669741196593856),
 ('paises vecinos', 0.0073096626863447195)]

In [85]:
data[data.cluster_topic==similar_topics[1]].sample(5)

,_id,label,a_code,text,target,cluster_topic
1919,1130585348046839810,servpub,NaN,"el gobierno no se ha pronunciado para nada, so...",1,6
1498,1231069036638851077,-1,SLV,no se que habra pasado con migracion en el aie...,-1,6
1307,1072240107447566336,-1,PRY,no importa que tantas pruebas hayan sobre el p...,-1,6
348,1085246117812596737,-1,PRY,"bueno, yo no estoy por alla, y los que me cono...",-1,6
1744,1072223587698905090,-1,ARG,"bulgaria ""en esta etapa, el gobierno bulgaro c...",-1,6
